# Testing mood packages

In [1]:
# IMPORT

import warnings

warnings.filterwarnings("ignore")

import datasets
import numpy as np
from transformers import pipeline

# micromamba install -c conda-forge \
#   pandas \
#   matplotlib \
#   pytorch \
#   tqdm \
#   keras=2.*

# pip install --upgrade \
#   transformers \
#   datasets

# micromamba install -c jupyterlab
# micromamba install -c conda-forge pytorch
# micromamba install -c conda-forge tensorflow
# micromamba remove keras
# micromamba install -c conda-forge keras=2.*

In [2]:
split_name = "test"

dataset_name, dataset_config_name = "go_emotions", "simplified"
dataset_dict = datasets.load_dataset(dataset_name, dataset_config_name)
dataset_dict[split_name][0]

{'text': 'I’m really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!',
 'labels': [25],
 'id': 'eecwqtt'}

In [3]:
labels = dataset_dict[split_name].features["labels"].feature.names
print({i: l for i, l in enumerate(labels)})
# {0: 'admiration', 1: 'amusement', 2: 'anger', 3: 'annoyance', 4: 'approval', 5: 'caring', 6: 'confusion',
# 7: 'curiosity', 8: 'desire', 9: 'disappointment', 10: 'disapproval', 11: 'disgust', 12: 'embarrassment',
# 13: 'excitement', 14: 'fear', 15: 'gratitude', 16: 'grief', 17: 'joy', 18: 'love', 19: 'nervousness',
# 20: 'optimism', 21: 'pride', 22: 'realization', 23: 'relief', 24: 'remorse', 25: 'sadness', 26: 'surprise', 27: 'neutral'}

{0: 'admiration', 1: 'amusement', 2: 'anger', 3: 'annoyance', 4: 'approval', 5: 'caring', 6: 'confusion', 7: 'curiosity', 8: 'desire', 9: 'disappointment', 10: 'disapproval', 11: 'disgust', 12: 'embarrassment', 13: 'excitement', 14: 'fear', 15: 'gratitude', 16: 'grief', 17: 'joy', 18: 'love', 19: 'nervousness', 20: 'optimism', 21: 'pride', 22: 'realization', 23: 'relief', 24: 'remorse', 25: 'sadness', 26: 'surprise', 27: 'neutral'}


In [4]:
num_items, num_labels = len(dataset_dict[split_name]), len(labels)
y_targets_all = np.zeros((num_items, num_labels), dtype=int)
for i, labels_indices in enumerate(dataset_dict[split_name]["labels"]):
    for label_index in labels_indices:
        y_targets_all[i, label_index] = 1

print(y_targets_all[0:3])

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


## OK Clear --

In [5]:
# Roberta Model Classification

# admiration
# amusement
# anger
# annoyance
# approval
# caring
# confusion
# curiosity
# desire
# disappointment
# disapproval
# disgust
# embarrassment
# excitement
# fear
# gratitude
# grief
# joy
# love
# nervousness
# optimism
# pride
# realization
# relief
# remorse
# sadness
# surprise
# neutral

classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

Device set to use mps:0


In [7]:
texts = list(dataset_dict[split_name]["text"])   # force python list[str]
model_outputs = classifier(texts)

print(dataset_dict[split_name]["text"][0])
print(model_outputs[0])

I’m really sorry about your situation :( Although I love the names Sapphira, Cirilla, and Scarlett!
[{'label': 'remorse', 'score': 0.678300678730011}, {'label': 'sadness', 'score': 0.5406177639961243}, {'label': 'love', 'score': 0.3608134984970093}, {'label': 'caring', 'score': 0.05576629564166069}, {'label': 'disappointment', 'score': 0.03692128136754036}, {'label': 'approval', 'score': 0.03364572301506996}, {'label': 'gratitude', 'score': 0.031087171286344528}, {'label': 'admiration', 'score': 0.018729237839579582}, {'label': 'grief', 'score': 0.018079757690429688}, {'label': 'optimism', 'score': 0.015966985374689102}, {'label': 'disapproval', 'score': 0.015757182613015175}, {'label': 'realization', 'score': 0.013122469186782837}, {'label': 'desire', 'score': 0.011346084997057915}, {'label': 'curiosity', 'score': 0.010657974518835545}, {'label': 'confusion', 'score': 0.01022509764879942}, {'label': 'anger', 'score': 0.00808509811758995}, {'label': 'embarrassment', 'score': 0.00747948

## Turn to better scores

In [8]:
y_probas_all = np.zeros(
    (num_items, num_labels),
    dtype=float
)

for i, item_probas in enumerate(model_outputs):
    for item_proba in item_probas:
        label, score = item_proba["label"], item_proba["score"]
        label_index = labels.index(label)
        y_probas_all[i, label_index] = score

In [9]:
i = 3856
print(dataset_dict[split_name][i])
print(np.round(y_probas_all[i], 3).tolist())
top = np.argmax(y_probas_all[i])
print(f"Top label proba is label number {top} ({labels[top]}): {y_probas_all[i][top]}")

{'text': 'Damn, still hoping...', 'labels': [20], 'id': 'edm34qc'}
[0.002, 0.003, 0.008, 0.022, 0.011, 0.015, 0.002, 0.002, 0.052, 0.01, 0.01, 0.002, 0.001, 0.002, 0.002, 0.005, 0.001, 0.002, 0.002, 0.001, 0.824, 0.001, 0.007, 0.001, 0.003, 0.003, 0.002, 0.157]
Top label proba is label number 20 (optimism): 0.8238562345504761


In [29]:
from math import floor


def floor_to_multiple(number, multiple = 0.5):
    return floor(number / multiple) * multiple

listy = [5,4,3,2,4,2,3,1,5]

floor_to_multiple(sum(listy) / len(listy))

3.0

In [30]:
# INPUT

sentences = ["I am not having a great day"]

In [31]:
# ROBERTA BASE

classifier = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

model_outputs = classifier(sentences)
print(model_outputs[0])

Device set to use mps:0


[{'label': 'disappointment', 'score': 0.4666956663131714}, {'label': 'sadness', 'score': 0.398494690656662}, {'label': 'annoyance', 'score': 0.0680660828948021}, {'label': 'neutral', 'score': 0.05703021585941315}, {'label': 'disapproval', 'score': 0.044239409267902374}, {'label': 'nervousness', 'score': 0.014850744046270847}, {'label': 'realization', 'score': 0.014059898443520069}, {'label': 'approval', 'score': 0.011267454363405704}, {'label': 'joy', 'score': 0.006303384434431791}, {'label': 'remorse', 'score': 0.006221487186849117}, {'label': 'caring', 'score': 0.006029391661286354}, {'label': 'embarrassment', 'score': 0.005265495739877224}, {'label': 'anger', 'score': 0.0049814400263130665}, {'label': 'disgust', 'score': 0.004259037785232067}, {'label': 'grief', 'score': 0.004002134781330824}, {'label': 'confusion', 'score': 0.003382924711331725}, {'label': 'relief', 'score': 0.0031404944602400064}, {'label': 'desire', 'score': 0.0028274687938392162}, {'label': 'admiration', 'score'